# Data exploration and ETL development using `BigQuery SQL` and `pandas` with BQ Dataframes

## Data exploration: Calculate average departure delay per airport

### BigQuery SQL

To use BigQuery SQL, simple add a new cell and click on the BigQuery icon.
You can also click on the BigQuery logo on the left menu and execute the query from the interacive SQL Editor.

#@bigquery
SELECT
    a.airport_name,
    AVG(f.departure_delay) AS average_departure_delay
FROM
    `airline.fact_flight` AS f
JOIN
    `airline.dim_flight` AS df ON f.flight_key = df.flight_key
JOIN
    `airline.dim_airport` AS a ON df.departure_airport_key = a.airport_key
GROUP BY
    a.airport_name
ORDER BY
    average_departure_delay DESC;

Alternatively, you can use the BigQuery python SDK

In [ ]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

dataset_id = "airline" 
project_id = "velascoluis-dev-sandbox"

query = f"""
SELECT
    a.airport_name,
    AVG(f.departure_delay) AS average_departure_delay
FROM
    `{project_id}.{dataset_id}.fact_flight` AS f
JOIN
    `{project_id}.{dataset_id}.dim_flight` AS df ON f.flight_key = df.flight_key
JOIN
    `{project_id}.{dataset_id}.dim_airport` AS a ON df.departure_airport_key = a.airport_key
GROUP BY
    a.airport_name
ORDER BY
    average_departure_delay DESC;
"""
df = client.query_and_wait(query).to_dataframe()
print(df)

### pandas (BQ Dataframes)

BigQuery Dataframes is an open source python packgage that transpiles from pandas syntax to scalable BQ SQL

In [ ]:
import bigframes.pandas as bpd


fact_flight_df = bpd.read_gbq("select * from airline.fact_flight")
dim_flight_df = bpd.read_gbq("select * from airline.dim_flight")
dim_airport_df = bpd.read_gbq("select * from airline.dim_airport")

joined_df = fact_flight_df.set_index('flight_key').join(dim_flight_df.set_index('flight_key'))
joined_df = joined_df.set_index('departure_airport_key').join(dim_airport_df.set_index('airport_key'))


average_delay_df = joined_df.groupby('airport_name')['departure_delay'].mean().reset_index()
average_delay_df = average_delay_df.rename(columns={'departure_delay': 'average_departure_delay'})
average_delay_df = average_delay_df.sort_values(by='average_departure_delay', ascending=False)

print(average_delay_df)

## ETL development: Airport delays aggregation

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()
# Step 1: Calculate total delay and on-time performance flag
query_step_1 = """CREATE OR REPLACE TABLE airline.etl_step_1_delays_sql AS
SELECT
    f.flight_key,
    COALESCE(f.departure_delay, 0) + COALESCE(f.arrival_delay, 0) AS total_delay,
    CASE WHEN COALESCE(f.departure_delay, 0) + COALESCE(f.arrival_delay, 0) <= 0 THEN 1 ELSE 0 END AS on_time_performance
FROM
    `velascoluis-dev-sandbox.airline.fact_flight` AS f;"""

# Step 2: Join with dim_flight and dim_airport to get airport names
query_step_2 = """CREATE OR REPLACE  TABLE airline.etl_step_2_flight_delays_with_airports_sql AS
SELECT
    d.total_delay,
    d.on_time_performance,
    a.airport_name
FROM
    airline.etl_step_1_delays_sql AS d
JOIN
    `velascoluis-dev-sandbox.airline.dim_flight` AS df ON d.flight_key = df.flight_key
JOIN
    `velascoluis-dev-sandbox.airline.dim_airport` AS a ON df.departure_airport_key = a.airport_key;"""

# Step 3: Aggregate by airport to calculate average delay and on-time percentage
query_step_3 = """CREATE OR REPLACE TABLE airline.etl_step_3_airport_performance_sql AS
SELECT
    a.airport_name,
    AVG(a.total_delay) AS average_total_delay,
    AVG(a.on_time_performance) AS on_time_percentage
FROM
    airline.etl_step_2_flight_delays_with_airports_sql AS a
GROUP BY
    a.airport_name;"""

In [ ]:
client.query_and_wait(query_step_1)
client.query_and_wait(query_step_2)
client.query_and_wait(query_step_3)

In [ ]:
client.query_and_wait("SELECT * FROM airline.etl_step_3_airport_performance_sql").to_dataframe()